In [20]:
print("En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo \
      \n el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde \
      \n a la cantidad de tareas realizadas en un tiempo determinado. \
      \n A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.")

En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo       
 el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde       
 a la cantidad de tareas realizadas en un tiempo determinado.       
 A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.


In [21]:
print("Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería \
      \n nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos \
      \n en algunos de los métodos que ya la librería implementa,  \
      \n sin embargo, es posible que también utilicemos otras liberías como AgentPy \
      \n puesto que en Mesa no estará todo lo que necesitamos ")

Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería       
 nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos       
 en algunos de los métodos que ya la librería implementa,        
 sin embargo, es posible que también utilicemos otras liberías como AgentPy       
 puesto que en Mesa no estará todo lo que necesitamos 


In [46]:
print("Estas son las importaciones de las librerías que utilizaremos:")

#Importaciones

import numpy
from mesa import Agent, Model
from mesa.space import MultiGrid
import agentpy as ap
import ipysimulate as ips
import ipysimulate as ips
from agentpy.examples import WealthModel, SegregationModel
import pandas as pd
from anytree import NodeMixin, RenderTree
from pprint import pprint as pp
import inspect

Estas son las importaciones de las librerías que utilizaremos:


In [6]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características

class ScheduleAgent:
  def __init__(self, name, age, gender, dailyTask, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.dailyTask = dailyTask
    self.notification = comments
    self.comments = comments

taskList = pd.read_csv('taskList.csv')
comentarios = ""

my_Agent = ScheduleAgent("Pepito", "20", "Masculino", taskList, comentarios)

In [8]:
# Estructura del árbol con el uso de la librería anytree

class Task:
    def __init__(self, id, name, avgTime, urgency):
        self.id=id
        self.name=name
        self.avgTime=avgTime
        self.urgency=urgency

class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self, name, avgTime, urgency, parent=None, children=None):
        super(Task, self).__init__()
        self.name = name
        self.avgTime = avgTime
        self.urgency = urgency
        self.parent = parent
        if children:
            self.children = children
    def __str__(self):
        print (self.name)


In [9]:
def buildTree(taskList:pd.DataFrame) -> TaskClass:
    nodeList = []
    for i in range(0, len(taskList["Name"])): 
        nodeList.append(TaskClass(
            name = taskList["Name"][i],
            avgTime = taskList["AvgTime"][i],
            urgency = taskList["Urgency"][i]))
        if i > 0:
            node = nodeList[i]
            node.parent = nodeList[(int(taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

In [10]:
def printTree(rootNode: TaskClass) -> None:
    for pre, fill, node in RenderTree(rootNode):
        print("%s%s" % (pre, node.name))
    

In [21]:
task_df = pd.read_csv('taskList.csv') 
task_tree = buildTree(task_df)

# print(task_df)
printTree(task_tree)

desayunar
├── bañarse
│   ├── hacer la tarea
│   │   ├── almorzar
│   │   └── hacer ejercicio
│   └── estudiar para el parcial
│       ├── escuchar musica
│       └── leer documentacion
└── sacar al perro
    ├── hacer estiramientos
    │   ├── ver netflix
    │   └── reunirse con amigos
    └── regar plantas
        ├── ordenar el cuarto
        └── jugar con el perro


In [66]:
# root
print("task tree: " + str(task_tree))
print("task tree.name: " + task_tree.name)
print("task tree.children: " + str(task_tree.children))
print("task tree.children left: " + str(task_tree.children[0])) #IZQ
print("task tree.children right: " + str(task_tree.children[1])) #DER
pp(task_tree.children)
print(type(task_tree.children))
print("task tree.height: " + str(task_tree.height))
print("task tree.root: " + str(task_tree.root))
print("task tree.children left name: " + str(task_tree.children[0].name)) #IZQ
print("task tree.children left name: " + str(task_tree.children[0].children[0])) #IZQ

task tree: <__main__.TaskClass object at 0x0000023A23993F90>
task tree.name: desayunar
task tree.children: (<__main__.TaskClass object at 0x0000023A239980D0>, <__main__.TaskClass object at 0x0000023A23992D10>)
task tree.children left: <__main__.TaskClass object at 0x0000023A239980D0>
task tree.children right: <__main__.TaskClass object at 0x0000023A23992D10>
(<__main__.TaskClass object at 0x0000023A239980D0>,
 <__main__.TaskClass object at 0x0000023A23992D10>)
<class 'tuple'>
task tree.height: 3
task tree.root: <__main__.TaskClass object at 0x0000023A23993F90>
task tree.children left name: bañarse
task tree.children left name: <__main__.TaskClass object at 0x0000023A2392E1D0>


In [ ]:
#arbol = task_tree

class Choice():
    def __init__(self, move, value):
        self.move = move
        self.value = value

    def __str__(self):
        return self.move + ": " + str(self.value)

# initialize our state
current_node = task_tree # iniciamos con la raiz
is_max = True

def minimax(node, is_max):    
    # base case, if no sub nodes, return the value
    if (node.left is None and node.right is None):
        return Choice("end", node.value)

    # if child nodes exist, run minimax on each child nodes
    l_choice = minimax(node.left, not is_max)
    r_choice = minimax(node.right, not is_max)
    
    # compare results
    if (is_max):
        if (l_choice.value > r_choice.value):
            return Choice("left", l_choice.value)
        else:
            return Choice("right", r_choice.value)
    else:
        if (l_choice.value < r_choice.value):
            return Choice("left", l_choice.value)
        else:
            return Choice("right", r_choice.value)
        
        
# run core programmmm


while (True):
    # run minimax on current node
    choice = minimax(current_node, is_max)
    
    # make choice based on minimax search
    if (choice.move == "left"):
        print ("Moving left to node with value " + str(current_node.left.value))
        current_node = current_node.left
    elif (choice.move == "right"):
        print ("Moving right to node with value " + str(current_node.right.value))
        current_node = current_node.right
    elif (choice.move == "end"):
        print ("Game ends with a score of " + str(choice.value))
        break